In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import geopandas as gpd
import folium
import json
from folium.plugins import HeatMap
from shapely.geometry import MultiPoint, Point
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import f1_score
import inspect

In [2]:
df = pd.read_csv('Access_to_Everyday_Life_Dataset.csv')
df.rename(columns = {'geometry/coordinates/0':'lon','geometry/coordinates/1':'lat', 'properties/attribute_id':'att_id', 
                     'properties/label_type':'label','properties/neighborhood':'neighborhood','properties/severity':'severity',
                    'properties/is_temporary':'is_temp'},inplace = True)


## Join geographic data with Everyday Life Dataset

In [3]:
# 2. Load the GeoJSON file
neighborhoods_gdf = gpd.read_file('Neighborhood_Map_Atlas_Neighborhoods.geojson')

# 3. Convert your DataFrame into a GeoDataFrame
# We create a 'geometry' column using the longitude and latitude
geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
incidents_gdf = gpd.GeoDataFrame(df, geometry=geometry)

# Set the Coordinate Reference System (CRS)
# GeoJSON snippet uses 'CRS84' (EPSG:4326): standard Lat/Lon
incidents_gdf.set_crs(epsg=4326, inplace=True)
neighborhoods_gdf.to_crs(epsg=4326, inplace=True)

# 5. Perform the Spatial Join
# This checks which neighborhood polygon each incident point "intersects"
# We only bring over 'S_HOOD' (Specific Neighborhood) and 'L_HOOD' (Larger Region)
joined_df = gpd.sjoin(
    incidents_gdf, 
    neighborhoods_gdf[['S_HOOD', 'L_HOOD', 'geometry']], 
    how='left', 
    predicate='intersects'
)

# 6. Final Clean up
# Rename 'S_HOOD' to 'neighborhood_accurate' and remove the geometry column if no longer needed
joined_df = joined_df.rename(columns={'S_HOOD': 'neighborhood_accurate'})
df_final = pd.DataFrame(joined_df.drop(columns='geometry'))

print(df_final[['lon', 'lat', 'neighborhood_accurate']].head())

          lon        lat neighborhood_accurate
0 -122.298981  47.594616              Atlantic
1 -122.301071  47.593357              Atlantic
2 -122.301079  47.596844              Atlantic
3 -122.301071  47.596500              Atlantic
4 -122.306274  47.599930              Atlantic


In [4]:
df_final.drop(columns = ['type','geometry/type'],inplace = True)

In [5]:
df_final.head()

,lon,lat,att_id,label,neighborhood,severity,is_temp,index_right,neighborhood_accurate,L_HOOD
0,-122.298981,47.594616,52096165,SurfaceProblem,Atlantic,4.0,False,31.0,Atlantic,Central Area
1,-122.301071,47.593357,52096166,SurfaceProblem,Atlantic,3.0,False,31.0,Atlantic,Central Area
2,-122.301079,47.596844,52096167,SurfaceProblem,Atlantic,4.0,False,31.0,Atlantic,Central Area
3,-122.301071,47.596500,52096168,SurfaceProblem,Atlantic,4.0,False,31.0,Atlantic,Central Area
4,-122.306274,47.599930,52096365,NoCurbRamp,Atlantic,4.0,False,31.0,Atlantic,Central Area


In [6]:
geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
gdf_incidents = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

In [7]:
# 3. Join with Official Neighborhood Atlas
neighborhoods_atlas = gpd.read_file('Neighborhood_Map_Atlas_Neighborhoods.geojson').to_crs("EPSG:4326")

gdf_with_hoods = gpd.sjoin(
    gdf_incidents, 
    neighborhoods_atlas[['S_HOOD', 'L_HOOD', 'geometry']], 
    how='left', 
    predicate='within'
)

# --- CRITICAL FIXES FOR DOUBLE JOINING ---
# 1. Rename the neighborhood column
gdf_with_hoods = gdf_with_hoods.rename(columns={'S_HOOD': 'neighborhood_accurate'})

# 2. Drop the join index so the next sjoin doesn't conflict
if 'index_right' in gdf_with_hoods.columns:
    gdf_with_hoods = gdf_with_hoods.drop(columns=['index_right'])

# 3. Reset the index to ensure it's clean for the next operation
gdf_with_hoods = gdf_with_hoods.reset_index(drop=True)
# -----------------------------------------

# 4. Join with ACS Demographic Data
url = "https://data-seattlecitygis.opendata.arcgis.com/datasets/SeattleCityGIS::seattle-neighborhoods-top-50-american-community-survey-data.geojson"
acs_gdf = gpd.read_file(url)

acs_cols = [
    'NEIGH_NAME', 'TOTAL_POPULATION', 'TOTAL_HOUSEHOLDS', 'Children_under_5',
    'Children_under_18', 'Older_Adults_65_over', 'Median_Age', 'Male', 'Female',
    'PEOPLE_OF_COLOR_PERCENT', 'BACHELOR_HIGHER_PERCENT', 'PER_CAPITA_INCOME',
    'RENTER_HOUSEHOLDS_PERCENT', 'DETACHED_1_UNIT_PERCENT',
    'PUBLIC_TRANSPORTATION_PERCENT', 'POPULATION_DISABILITY_PERC',
    'geometry'
]
acs_gdf_small = acs_gdf[acs_cols].to_crs(gdf_with_hoods.crs)

# Perform the second spatial join
merged_final_gdf = gpd.sjoin(
    gdf_with_hoods,
    acs_gdf_small,
    how="left",
    predicate="within"
)

# 5. Final Cleanup
# If you want to keep it as a dataframe for ML models:
df_final_clean = pd.DataFrame(merged_final_gdf.drop(columns='geometry'))

print(f"Final Shape: {df_final_clean.shape}")
print(df_final_clean[['neighborhood_accurate', 'NEIGH_NAME', 'TOTAL_POPULATION']].head())

Final Shape: (163893, 28)
  neighborhood_accurate            NEIGH_NAME  TOTAL_POPULATION
0              Atlantic    Council District 3          108012.0
0              Atlantic  23rd & Union-Jackson           16724.0
1              Atlantic    Council District 3          108012.0
1              Atlantic  23rd & Union-Jackson           16724.0
2              Atlantic    Council District 3          108012.0


In [8]:
df_final_clean.head()

,type,geometry/type,lon,lat,att_id,label,neighborhood,severity,is_temp,neighborhood_accurate,...,Median_Age,Male,Female,PEOPLE_OF_COLOR_PERCENT,BACHELOR_HIGHER_PERCENT,PER_CAPITA_INCOME,RENTER_HOUSEHOLDS_PERCENT,DETACHED_1_UNIT_PERCENT,PUBLIC_TRANSPORTATION_PERCENT,POPULATION_DISABILITY_PERC
0,Feature,Point,-122.298981,47.594616,52096165,SurfaceProblem,Atlantic,4.0,False,Atlantic,...,36.4,56878.0,51134.0,35.9,74.6,99655.0,65.7,23.6,17.1,12.4
0,Feature,Point,-122.298981,47.594616,52096165,SurfaceProblem,Atlantic,4.0,False,Atlantic,...,36.3,8580.0,8144.0,51.7,61.9,70278.0,58.4,29.8,16.3,17.2
1,Feature,Point,-122.301071,47.593357,52096166,SurfaceProblem,Atlantic,3.0,False,Atlantic,...,36.4,56878.0,51134.0,35.9,74.6,99655.0,65.7,23.6,17.1,12.4
1,Feature,Point,-122.301071,47.593357,52096166,SurfaceProblem,Atlantic,3.0,False,Atlantic,...,36.3,8580.0,8144.0,51.7,61.9,70278.0,58.4,29.8,16.3,17.2
2,Feature,Point,-122.301079,47.596844,52096167,SurfaceProblem,Atlantic,4.0,False,Atlantic,...,36.4,56878.0,51134.0,35.9,74.6,99655.0,65.7,23.6,17.1,12.4


In [9]:
df_final_clean.columns

Index(['type', 'geometry/type', 'lon', 'lat', 'att_id', 'label',
       'neighborhood', 'severity', 'is_temp', 'neighborhood_accurate',
       'L_HOOD', 'index_right', 'NEIGH_NAME', 'TOTAL_POPULATION',
       'TOTAL_HOUSEHOLDS', 'Children_under_5', 'Children_under_18',
       'Older_Adults_65_over', 'Median_Age', 'Male', 'Female',
       'PEOPLE_OF_COLOR_PERCENT', 'BACHELOR_HIGHER_PERCENT',
       'PER_CAPITA_INCOME', 'RENTER_HOUSEHOLDS_PERCENT',
       'DETACHED_1_UNIT_PERCENT', 'PUBLIC_TRANSPORTATION_PERCENT',
       'POPULATION_DISABILITY_PERC'],
      dtype='object')

In [10]:
df_final_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 163893 entries, 0 to 81972
Data columns (total 28 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   type                           163893 non-null  object 
 1   geometry/type                  163893 non-null  object 
 2   lon                            163893 non-null  float64
 3   lat                            163893 non-null  float64
 4   att_id                         163893 non-null  int64  
 5   label                          163893 non-null  object 
 6   neighborhood                   163893 non-null  object 
 7   severity                       159392 non-null  float64
 8   is_temp                        163893 non-null  bool   
 9   neighborhood_accurate          163840 non-null  object 
 10  L_HOOD                         163840 non-null  object 
 11  index_right                    163840 non-null  float64
 12  NEIGH_NAME                     16384

In [11]:
len(df_final)/len(df_final_clean)

0.5001616908592802

In [12]:
check_1 = df_final_clean[df_final_clean['neighborhood_accurate'] == df_final_clean['NEIGH_NAME']]
len(check_1)/len(df_final_clean)

0.04488294191942304

In [13]:
df_final_clean['NEIGH_NAME'].head()

0      Council District 3
0    23rd & Union-Jackson
1      Council District 3
1    23rd & Union-Jackson
2      Council District 3
Name: NEIGH_NAME, dtype: object

In [14]:
df_final_clean['L_HOOD']

0        Central Area
0        Central Area
1        Central Area
1        Central Area
2        Central Area
             ...     
81970    Central Area
81971    Central Area
81971    Central Area
81972    Central Area
81972    Central Area
Name: L_HOOD, Length: 163893, dtype: object

In [15]:
# Drop na values
df_labeled = df_final_clean.dropna(subset=['severity'])
# Pre-process 'is_temp' to be 1 and 0
df_labeled['is_temp'] = df_labeled['is_temp'].astype(int)
features = [ 'lon', 'lat', 'att_id', 'label','is_temp', 'neighborhood_accurate', 'TOTAL_POPULATION',
       'TOTAL_HOUSEHOLDS', 'Children_under_5', 'Children_under_18',
       'Older_Adults_65_over', 'Median_Age', 'Male', 'Female',
       'PEOPLE_OF_COLOR_PERCENT', 'BACHELOR_HIGHER_PERCENT',
       'PER_CAPITA_INCOME', 'RENTER_HOUSEHOLDS_PERCENT',
       'DETACHED_1_UNIT_PERCENT', 'PUBLIC_TRANSPORTATION_PERCENT',
       'POPULATION_DISABILITY_PERC']
target = 'severity'
# Drop all remaining nas - those rows are negligible
df_labeled.dropna(inplace = True)
X = df_labeled[features]
y = df_labeled[target]



/var/folders/58/bxfs02q56pzfmzl9zs71htz00000gn/T/ipykernel_12406/3154352185.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_labeled['is_temp'] = df_labeled['is_temp'].astype(int)
/var/folders/58/bxfs02q56pzfmzl9zs71htz00000gn/T/ipykernel_12406/3154352185.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_labeled.dropna(inplace = True)


In [16]:
# 3. Create a Stratification Key
# This combines severity and neighborhood to ensure the split is proportional for both.
# We fill NAs in categorical columns temporarily to prevent errors in concatenation.
stratify_cols = ['severity', 'neighborhood_accurate']
stratify_key = df_labeled[stratify_cols].astype(str).agg('-'.join, axis=1)

# Handle cases where a neighborhood/severity combo appears only once 
# (Stratify requires at least 2 members per group)
counts = stratify_key.value_counts()
valid_indices = stratify_key.isin(counts[counts > 1].index)

X_final = X[valid_indices]
y_final = y[valid_indices]
stratify_final = stratify_key[valid_indices] 

In [17]:
# 4. Perform the Split (80% Train, 20% Test)
X_train, X_test, y_train, y_test = train_test_split(
    X_final, 
    y_final, 
    test_size=0.20, 
    random_state=42, 
    stratify=stratify_final # This keeps neighborhood and severity proportions identical
)

print(f"Training set size: {len(X_train)}")
print(f"Testing set size: {len(X_test)}")

Training set size: 126773
Testing set size: 31694


In [18]:


# 2. Re-run the split logic from the previous step 
# (Assuming X_train, X_test, y_train, y_test are defined as per the previous stratified split)

# 3. Define the ColumnTransformer
# This will OneHotEncode 'label' and 'neighborhood_accurate'
# 'passthrough' means 'is_temp' will be kept as is (1s and 0s)
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['label', 'neighborhood_accurate'])
    ],
    remainder='passthrough'
)

In [19]:
# 4. Create a Pipeline
# A pipeline bundles the preprocessing and the model together.
# This prevents data leakage and makes it easier to predict on new data.
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# 5. Train the model
clf.fit(X_train, y_train)

# 6. Evaluate
print(f"Model Accuracy on Test Set: {clf.score(X_test, y_test):.2%}")

Model Accuracy on Test Set: 72.06%


## XGBoost Model

can't have labels starting from 1.0: use labelencoder

In [20]:
df_labeled2 = df_final_clean.dropna(subset=['severity']).copy()
df_labeled2['is_temp'] = df_labeled2['is_temp'].astype(int)


In [21]:
le = LabelEncoder()
y = le.fit_transform(df_labeled2['severity']) 

# 3. Define Features
# One-hot encoding neighborhood and label; keeping is_temp as binary
features = [ 'lon', 'lat', 'att_id', 'label','is_temp', 'neighborhood_accurate', 'TOTAL_POPULATION',
       'TOTAL_HOUSEHOLDS', 'Children_under_5', 'Children_under_18',
       'Older_Adults_65_over', 'Median_Age', 'Male', 'Female',
       'PEOPLE_OF_COLOR_PERCENT', 'BACHELOR_HIGHER_PERCENT',
       'PER_CAPITA_INCOME', 'RENTER_HOUSEHOLDS_PERCENT',
       'DETACHED_1_UNIT_PERCENT', 'PUBLIC_TRANSPORTATION_PERCENT',
       'POPULATION_DISABILITY_PERC']

X = df_labeled2[features].copy()
X['is_temp'] = X['is_temp'].astype(int)

# 4. Stratified Split (Ensures 1.0-5.0 distribution is same in train and test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 5. Pipeline with Preprocessing and XGBoost
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['label', 'neighborhood_accurate'])
    ],
    remainder='passthrough'
)

xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        objective='multi:softprob', # Predicts probability for each of the 5 buckets
        num_class=5,                # Explicitly setting 5 classes
        random_state=42
    ))
])

# 6. Train the Model
xgb_pipeline.fit(X_train, y_train)

# 7. Evaluate Performance
y_pred = xgb_pipeline.predict(X_test)

# Convert back to original 1.0-5.0 scale for the report
print("Classification Report (Original Severity Scales):")
print(classification_report(le.inverse_transform(y_test), le.inverse_transform(y_pred)))

Classification Report (Original Severity Scales):
              precision    recall  f1-score   support

         1.0       0.64      0.84      0.72      7056
         2.0       0.58      0.40      0.47      5844
         3.0       0.61      0.75      0.67      9281
         4.0       0.81      0.52      0.63      5582
         5.0       0.73      0.65      0.69      4116

    accuracy                           0.65     31879
   macro avg       0.67      0.63      0.64     31879
weighted avg       0.66      0.65      0.64     31879



### Running several XGBoost Models, with parameters features. 
Implement feature subset selection loop - begin with 0 features, iteratively add features that increase Weighted F1-score the most until target limit is reached. 

In [22]:
# Use full list of candidate features from before
all_features = [
    'lon', 'lat', 'label', 'is_temp', 'neighborhood_accurate', 
    'TOTAL_POPULATION', 'TOTAL_HOUSEHOLDS', 'Children_under_5', 'Children_under_18',
    'Older_Adults_65_over', 'Median_Age', 'Male', 'Female',
    'PEOPLE_OF_COLOR_PERCENT', 'BACHELOR_HIGHER_PERCENT',
    'PER_CAPITA_INCOME', 'RENTER_HOUSEHOLDS_PERCENT',
    'DETACHED_1_UNIT_PERCENT', 'PUBLIC_TRANSPORTATION_PERCENT',
    'POPULATION_DISABILITY_PERC'
]

# Helper function to train and score a specific subset
def evaluate_subset(feature_subset):
    # Identify columns in this subset that need OneHotEncoding
    cat_cols = [f for f in feature_subset if f in ['label', 'neighborhood_accurate']]
    
    # Update preprocessor for only the categorical features present in this subset
    current_preprocessor = ColumnTransformer(
        transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)],
        remainder='passthrough'
    )
    
    current_pipeline = Pipeline(steps=[
        ('preprocessor', current_preprocessor),
        ('classifier', XGBClassifier(n_estimators=50, max_depth=5, learning_rate=0.1, 
                                     objective='multi:softprob', num_class=5, random_state=42))
    ])
    
    current_pipeline.fit(X_train[feature_subset], y_train)
    preds = current_pipeline.predict(X_test[feature_subset])
    return f1_score(y_test, preds, average='weighted')

In [23]:
# 2. Sequential Forward Selection Loop
best_features = []
remaining_features = all_features.copy()
best_score = 0
history = []

print("Starting Feature Optimization...")

# We will try to find the best subset up to N-1 features
for i in range(len(all_features) - 1):
    scores_in_this_round = []
    
    for feature in remaining_features:
        trial_features = best_features + [feature]
        score = evaluate_subset(trial_features)
        scores_in_this_round.append((score, feature))
    
    # Find the feature that gave the best improvement
    scores_in_this_round.sort(reverse=True)
    current_best_score, current_best_feature = scores_in_this_round[0]
    
    if current_best_score > best_score:
        best_score = current_best_score
        best_features.append(current_best_feature)
        remaining_features.remove(current_best_feature)
        history.append((len(best_features), best_score, best_features.copy()))
        print(f"Features: {len(best_features)} | Best Weighted F1: {best_score:.4f} | Added: {current_best_feature}")
    else:
        # If adding more features doesn't help, we stop early
        print("Stopping early: No further improvement.")
        break

print("\n--- OPTIMIZATION COMPLETE ---")
print(f"Optimal Number of Features: {len(best_features)}")
print(f"Best Features: {best_features}")

Starting Feature Optimization...
Features: 1 | Best Weighted F1: 0.3982 | Added: neighborhood_accurate
Features: 2 | Best Weighted F1: 0.5818 | Added: label
Features: 3 | Best Weighted F1: 0.5983 | Added: lat
Features: 4 | Best Weighted F1: 0.6054 | Added: lon
Features: 5 | Best Weighted F1: 0.6080 | Added: PEOPLE_OF_COLOR_PERCENT
Features: 6 | Best Weighted F1: 0.6089 | Added: POPULATION_DISABILITY_PERC
Stopping early: No further improvement.

--- OPTIMIZATION COMPLETE ---
Optimal Number of Features: 6
Best Features: ['neighborhood_accurate', 'label', 'lat', 'lon', 'PEOPLE_OF_COLOR_PERCENT', 'POPULATION_DISABILITY_PERC']


Program initially stopped early (after 5 features) - realized model is likely memoorizing features due to att_id. Geographic features may also limit impact of predictions. Initially removed "att_id" after first run-through.

Features after second run-through: 
Best Features: ['neighborhood_accurate', 'label', 'lat', 'lon', 'PEOPLE_OF_COLOR_PERCENT', 'POPULATION_DISABILITY_PERC']

Using features identified after second run-through. 

In [24]:
features = ['neighborhood_accurate', 'label', 'lat', 'lon', 'PEOPLE_OF_COLOR_PERCENT', 'POPULATION_DISABILITY_PERC']

X = df_labeled2[features].copy()
# X['is_temp'] = X['is_temp'].astype(int)

# 4. Stratified Split (Ensures 1.0-5.0 distribution is same in train and test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 5. Pipeline with Preprocessing and XGBoost
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['label', 'neighborhood_accurate'])
    ],
    remainder='passthrough'
)

xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        objective='multi:softprob', # Predicts probability for each of the 5 buckets
        num_class=5,                # Explicitly setting 5 classes
        random_state=42
    ))
])

# 6. Train the Model
xgb_pipeline.fit(X_train, y_train)

# 7. Evaluate Performance
y_pred = xgb_pipeline.predict(X_test)

# Convert back to original 1.0-5.0 scale for the report
print("Classification Report (Original Severity Scales):")
print(classification_report(le.inverse_transform(y_test), le.inverse_transform(y_pred)))

Classification Report (Original Severity Scales):
              precision    recall  f1-score   support

         1.0       0.64      0.84      0.73      7056
         2.0       0.59      0.39      0.47      5844
         3.0       0.61      0.75      0.67      9281
         4.0       0.80      0.53      0.64      5582
         5.0       0.74      0.64      0.69      4116

    accuracy                           0.65     31879
   macro avg       0.68      0.63      0.64     31879
weighted avg       0.66      0.65      0.64     31879



Running the code used above to evaluate models with different parameters, but seeing if there are better factors than lat and lon - XGBoost model may have started memorizing those values. 

In [25]:
# Use full list of candidate features from before
all_features = [
    'label', 'is_temp', 'neighborhood_accurate', 
    'TOTAL_POPULATION', 'TOTAL_HOUSEHOLDS', 'Children_under_5', 'Children_under_18',
    'Older_Adults_65_over', 'Median_Age', 'Male', 'Female',
    'PEOPLE_OF_COLOR_PERCENT', 'BACHELOR_HIGHER_PERCENT',
    'PER_CAPITA_INCOME', 'RENTER_HOUSEHOLDS_PERCENT',
    'DETACHED_1_UNIT_PERCENT', 'PUBLIC_TRANSPORTATION_PERCENT',
    'POPULATION_DISABILITY_PERC'
]

# Helper function to train and score a specific subset
def evaluate_subset(feature_subset):
    # Identify columns in this subset that need OneHotEncoding
    cat_cols = [f for f in feature_subset if f in ['label', 'neighborhood_accurate']]
    
    # Update preprocessor for only the categorical features present in this subset
    current_preprocessor = ColumnTransformer(
        transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)],
        remainder='passthrough'
    )
    
    current_pipeline = Pipeline(steps=[
        ('preprocessor', current_preprocessor),
        ('classifier', XGBClassifier(n_estimators=50, max_depth=5, learning_rate=0.1, 
                                     objective='multi:softprob', num_class=5, random_state=42))
    ])
    
    current_pipeline.fit(X_train[feature_subset], y_train)
    preds = current_pipeline.predict(X_test[feature_subset])
    return f1_score(y_test, preds, average='weighted')

In [26]:
# 1. Ensure you are using the correct dataframe that has ALL the columns
# (Using df_final_clean or your most recent merged dataframe)
df_working = df_final_clean.dropna(subset=['severity']).copy()

# 2. Make sure is_temp is an integer (XGBoost requirement)
if 'is_temp' in df_working.columns:
    df_working['is_temp'] = df_working['is_temp'].astype(int)

# 3. Re-define X and y with the full candidate list
X = df_working[all_features]
y = LabelEncoder().fit_transform(df_working['severity'])

# 4. Re-split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# NOW you can run your "Starting Feature Optimization..." loop

In [27]:
# 2. Sequential Forward Selection Loop
best_features = []
remaining_features = all_features.copy()
best_score = 0
history = []

print("Starting Feature Optimization...")

# We will try to find the best subset up to N-1 features
for i in range(len(all_features) - 1):
    scores_in_this_round = []
    
    for feature in remaining_features:
        trial_features = best_features + [feature]
        score = evaluate_subset(trial_features)
        scores_in_this_round.append((score, feature))
    
    # Find the feature that gave the best improvement
    scores_in_this_round.sort(reverse=True)
    current_best_score, current_best_feature = scores_in_this_round[0]
    
    if current_best_score > best_score:
        best_score = current_best_score
        best_features.append(current_best_feature)
        remaining_features.remove(current_best_feature)
        history.append((len(best_features), best_score, best_features.copy()))
        print(f"Features: {len(best_features)} | Best Weighted F1: {best_score:.4f} | Added: {current_best_feature}")
    else:
        # If adding more features doesn't help, we stop early
        print("Stopping early: No further improvement.")
        break

print("\n--- OPTIMIZATION COMPLETE ---")
print(f"Optimal Number of Features: {len(best_features)}")
print(f"Best Features: {best_features}")

Starting Feature Optimization...
Features: 1 | Best Weighted F1: 0.3982 | Added: neighborhood_accurate
Features: 2 | Best Weighted F1: 0.5818 | Added: label
Features: 3 | Best Weighted F1: 0.5916 | Added: RENTER_HOUSEHOLDS_PERCENT
Features: 4 | Best Weighted F1: 0.5925 | Added: PUBLIC_TRANSPORTATION_PERCENT
Features: 5 | Best Weighted F1: 0.5927 | Added: Older_Adults_65_over
Features: 6 | Best Weighted F1: 0.5933 | Added: TOTAL_POPULATION
Features: 7 | Best Weighted F1: 0.5933 | Added: Male
Stopping early: No further improvement.

--- OPTIMIZATION COMPLETE ---
Optimal Number of Features: 7
Best Features: ['neighborhood_accurate', 'label', 'RENTER_HOUSEHOLDS_PERCENT', 'PUBLIC_TRANSPORTATION_PERCENT', 'Older_Adults_65_over', 'TOTAL_POPULATION', 'Male']


Using features from analysis used above

In [28]:
features = ['neighborhood_accurate', 'label', 'RENTER_HOUSEHOLDS_PERCENT', 'PUBLIC_TRANSPORTATION_PERCENT', 'Older_Adults_65_over', 'TOTAL_POPULATION', 'Male']

X = df_labeled2[features].copy()
# X['is_temp'] = X['is_temp'].astype(int)

# 4. Stratified Split (Ensures 1.0-5.0 distribution is same in train and test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 5. Pipeline with Preprocessing and XGBoost
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['label', 'neighborhood_accurate'])
    ],
    remainder='passthrough'
)

xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        objective='multi:softprob', # Predicts probability for each of the 5 buckets
        num_class=5,                # Explicitly setting 5 classes
        random_state=42
    ))
])

# 6. Train the Model
xgb_pipeline.fit(X_train, y_train)

# 7. Evaluate Performance
y_pred = xgb_pipeline.predict(X_test)

# Convert back to original 1.0-5.0 scale for the report
print("Classification Report (Original Severity Scales):")
print(classification_report(le.inverse_transform(y_test), le.inverse_transform(y_pred)))

Classification Report (Original Severity Scales):
              precision    recall  f1-score   support

         1.0       0.60      0.84      0.70      7056
         2.0       0.57      0.31      0.41      5844
         3.0       0.59      0.74      0.66      9281
         4.0       0.81      0.48      0.60      5582
         5.0       0.67      0.65      0.66      4116

    accuracy                           0.63     31879
   macro avg       0.65      0.60      0.61     31879
weighted avg       0.64      0.63      0.61     31879



Created following models based on intuition:

In [29]:
features = ['neighborhood_accurate', 'label', 'lat', 'lon', 'TOTAL_POPULATION', 'is_temp']

X = df_labeled2[features].copy()
# X['is_temp'] = X['is_temp'].astype(int)

# 4. Stratified Split (Ensures 1.0-5.0 distribution is same in train and test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 5. Pipeline with Preprocessing and XGBoost
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['label', 'neighborhood_accurate'])
    ],
    remainder='passthrough'
)

xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        objective='multi:softprob', # Predicts probability for each of the 5 buckets
        num_class=5,                # Explicitly setting 5 classes
        random_state=42
    ))
])

# 6. Train the Model
xgb_pipeline.fit(X_train, y_train)

# 7. Evaluate Performance
y_pred = xgb_pipeline.predict(X_test)

# Convert back to original 1.0-5.0 scale for the report
print("Classification Report (Original Severity Scales):")
print(classification_report(le.inverse_transform(y_test), le.inverse_transform(y_pred)))

Classification Report (Original Severity Scales):
              precision    recall  f1-score   support

         1.0       0.64      0.84      0.73      7056
         2.0       0.60      0.39      0.48      5844
         3.0       0.61      0.75      0.67      9281
         4.0       0.80      0.53      0.63      5582
         5.0       0.73      0.64      0.69      4116

    accuracy                           0.65     31879
   macro avg       0.68      0.63      0.64     31879
weighted avg       0.66      0.65      0.64     31879



In [30]:
features = ['neighborhood_accurate', 'label', 'lat', 'lon', 'is_temp']

X = df_labeled2[features].copy()
# X['is_temp'] = X['is_temp'].astype(int)

# 4. Stratified Split (Ensures 1.0-5.0 distribution is same in train and test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 5. Pipeline with Preprocessing and XGBoost
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['label', 'neighborhood_accurate'])
    ],
    remainder='passthrough'
)

xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        objective='multi:softprob', # Predicts probability for each of the 5 buckets
        num_class=5,                # Explicitly setting 5 classes
        random_state=42
    ))
])

# 6. Train the Model
xgb_pipeline.fit(X_train, y_train)

# 7. Evaluate Performance
y_pred = xgb_pipeline.predict(X_test)

# Convert back to original 1.0-5.0 scale for the report
print("Classification Report (Original Severity Scales):")
print(classification_report(le.inverse_transform(y_test), le.inverse_transform(y_pred)))

Classification Report (Original Severity Scales):
              precision    recall  f1-score   support

         1.0       0.64      0.84      0.72      7056
         2.0       0.60      0.39      0.47      5844
         3.0       0.61      0.75      0.67      9281
         4.0       0.80      0.53      0.63      5582
         5.0       0.73      0.64      0.68      4116

    accuracy                           0.65     31879
   macro avg       0.67      0.63      0.64     31879
weighted avg       0.66      0.65      0.64     31879



In [31]:
features = ['neighborhood_accurate', 'label', 'lat', 'lon', 'is_temp','BACHELOR_HIGHER_PERCENT','PER_CAPITA_INCOME',
            'RENTER_HOUSEHOLDS_PERCENT','PUBLIC_TRANSPORTATION_PERCENT','POPULATION_DISABILITY_PERC','Older_Adults_65_over']

X = df_labeled2[features].copy()
# X['is_temp'] = X['is_temp'].astype(int)

# 4. Stratified Split (Ensures 1.0-5.0 distribution is same in train and test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 5. Pipeline with Preprocessing and XGBoost
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['label', 'neighborhood_accurate'])
    ],
    remainder='passthrough'
)

xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        objective='multi:softprob', # Predicts probability for each of the 5 buckets
        num_class=5,                # Explicitly setting 5 classes
        random_state=42
    ))
])

# 6. Train the Model
xgb_pipeline.fit(X_train, y_train)

# 7. Evaluate Performance
y_pred = xgb_pipeline.predict(X_test)

# Convert back to original 1.0-5.0 scale for the report
print("Classification Report (Original Severity Scales):")
print(classification_report(le.inverse_transform(y_test), le.inverse_transform(y_pred)))

Classification Report (Original Severity Scales):
              precision    recall  f1-score   support

         1.0       0.64      0.84      0.73      7056
         2.0       0.60      0.39      0.47      5844
         3.0       0.61      0.76      0.68      9281
         4.0       0.80      0.53      0.64      5582
         5.0       0.74      0.65      0.69      4116

    accuracy                           0.65     31879
   macro avg       0.68      0.63      0.64     31879
weighted avg       0.66      0.65      0.64     31879



In [32]:
all_features = [
    'label', 'is_temp', 'neighborhood_accurate', 
    'TOTAL_POPULATION', 'TOTAL_HOUSEHOLDS', 'Children_under_5', 'Children_under_18',
    'Older_Adults_65_over', 'Median_Age', 'Male', 'Female',
    'PEOPLE_OF_COLOR_PERCENT', 'BACHELOR_HIGHER_PERCENT',
    'PER_CAPITA_INCOME', 'RENTER_HOUSEHOLDS_PERCENT',
    'DETACHED_1_UNIT_PERCENT', 'PUBLIC_TRANSPORTATION_PERCENT',
    'POPULATION_DISABILITY_PERC'
]

In [34]:
features = ['neighborhood_accurate', 'label', 'lat', 'lon', 'is_temp']

X = df_labeled2[features].copy()
# X['is_temp'] = X['is_temp'].astype(int)

# 4. Stratified Split (Ensures 1.0-5.0 distribution is same in train and test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 5. Pipeline with Preprocessing and XGBoost
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['label', 'neighborhood_accurate'])
    ],
    remainder='passthrough'
)

xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        objective='multi:softprob', # Predicts probability for each of the 5 buckets
        num_class=5,                # Explicitly setting 5 classes
        random_state=42
    ))
])

# 6. Train the Model
xgb_pipeline.fit(X_train, y_train)

# 7. Evaluate Performance
y_pred = xgb_pipeline.predict(X_test)

# Convert back to original 1.0-5.0 scale for the report
print("Classification Report (Original Severity Scales):")
print(classification_report(le.inverse_transform(y_test), le.inverse_transform(y_pred)))

Classification Report (Original Severity Scales):
              precision    recall  f1-score   support

         1.0       0.64      0.84      0.72      7056
         2.0       0.60      0.39      0.47      5844
         3.0       0.61      0.75      0.67      9281
         4.0       0.80      0.53      0.63      5582
         5.0       0.73      0.64      0.68      4116

    accuracy                           0.65     31879
   macro avg       0.67      0.63      0.64     31879
weighted avg       0.66      0.65      0.64     31879



### Trying original non-joined dataset


In [35]:
# Trying original non-joined dataset

df_final.dropna(inplace= True)

In [40]:
df_final.isna().sum()

lon                      0
lat                      0
att_id                   0
label                    0
neighborhood             0
severity                 0
is_temp                  0
index_right              0
neighborhood_accurate    0
L_HOOD                   0
dtype: int64

In [42]:
all_features_og = [
    'lat','lon','is_temp', 'neighborhood_accurate'
]

X = df_final[all_features_og].copy()

le = LabelEncoder()
y = le.fit_transform(df_final['severity']) 
# X['is_temp'] = X['is_temp'].astype(int)

# 4. Stratified Split (Ensures 1.0-5.0 distribution is same in train and test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 5. Pipeline with Preprocessing and XGBoost
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['neighborhood_accurate'])
    ],
    remainder='passthrough'
)

xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        objective='multi:softprob', # Predicts probability for each of the 5 buckets
        num_class=5,                # Explicitly setting 5 classes
        random_state=42
    ))
])

# 6. Train the Model
xgb_pipeline.fit(X_train, y_train)

# 7. Evaluate Performance
y_pred = xgb_pipeline.predict(X_test)

# Convert back to original 1.0-5.0 scale for the report
print("Classification Report (Original Severity Scales):")
print(classification_report(le.inverse_transform(y_test), le.inverse_transform(y_pred)))

Classification Report (Original Severity Scales):
              precision    recall  f1-score   support

         1.0       0.46      0.53      0.49      3528
         2.0       0.40      0.19      0.26      2920
         3.0       0.45      0.66      0.53      4640
         4.0       0.68      0.43      0.52      2790
         5.0       0.56      0.53      0.55      2057

    accuracy                           0.49     15935
   macro avg       0.51      0.47      0.47     15935
weighted avg       0.50      0.49      0.47     15935



In [44]:
subset_features_og = [
   'neighborhood_accurate'
]

X = df_final[all_features_og].copy()

le = LabelEncoder()
y = le.fit_transform(df_final['severity']) 
# X['is_temp'] = X['is_temp'].astype(int)

# 4. Stratified Split (Ensures 1.0-5.0 distribution is same in train and test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 5. Pipeline with Preprocessing and XGBoost
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['neighborhood_accurate'])
    ],
    remainder='passthrough'
)

xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        objective='multi:softprob', # Predicts probability for each of the 5 buckets
        num_class=5,                # Explicitly setting 5 classes
        random_state=42
    ))
])

# 6. Train the Model
xgb_pipeline.fit(X_train, y_train)

# 7. Evaluate Performance
y_pred = xgb_pipeline.predict(X_test)

# Convert back to original 1.0-5.0 scale for the report
print("Classification Report (Original Severity Scales):")
print(classification_report(le.inverse_transform(y_test), le.inverse_transform(y_pred)))

Classification Report (Original Severity Scales):
              precision    recall  f1-score   support

         1.0       0.46      0.53      0.49      3528
         2.0       0.40      0.19      0.26      2920
         3.0       0.45      0.66      0.53      4640
         4.0       0.68      0.43      0.52      2790
         5.0       0.56      0.53      0.55      2057

    accuracy                           0.49     15935
   macro avg       0.51      0.47      0.47     15935
weighted avg       0.50      0.49      0.47     15935

